# DCGAN   - 비지도 학습
CNN을 GAN에 적용한 알고리즘

MNIST dataset을 사용할 것

input -> Generator(생성자) -> take -> Descriminator가 판별(원본과 구분할 수 없는 가짜 데이터가 생성될 때까지 반복)

In [ ]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Activation, Conv2D, UpSampling2D, LeakyReLU
from keras.models import Sequential, Model
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

In [ ]:
# Generator가 만드는 이미지 저장 폴더
if not os.path.exists('./gan_imgs'):
  os.makedirs('./gan_imgs')

# np.random.seed(3)
tf.random.set_seed(3)

# Generator model
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(alpha=0.2)))  # GAN은 ReLU 학습이 다소 불안정 하기에 LeakyReLU사용
generator.add(BatchNormalization())  # 데이터 정규화 - 안정적 학습 :과적합 방지 효과
generator.add(Reshape((7,7,128)))
generator.add(UpSampling2D())  # 이미지 크기를 2배 확장
generator.add(Conv2D(64, kernel_size=5, padding='same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.3)))  # alpha = 음의 기울기 계수  -1 ~ 1 사이
generator.add(UpSampling2D())  # 이미지 크기를 2배 확장
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))

print(generator.summary())

# Discriminator model
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, padding='same', strides=2, input_shape=(28, 28, 1)))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Conv2D(128, kernel_size=5, padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Flatten())
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation='sigmoid'))
print(discriminator.summary())

discriminator.compile(optimizer='adam', loss='binary_crossentropy')
discriminator.trainable = False

In [ ]:
# GAN 모델 생성
ginput = Input(shape=(100,))
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)

# GAN 모델 컴파일
gan.compile(optimizer='adam', loss='binary_crossentropy')

# GAN 모델 요약 출력
print(gan.summary())


In [ ]:
# 신경망 실행 함수
def gan_train(epoch, batch_size, saving_interval):
    (x_train, _),(x_test, _)= mnist.load_data()
    x_train = x_train.reshape(x_train.shape[0],28,28,1).astype('float32')
    x_train = (x_train-127.5)/127.5  #-1~1 사이 값으로 변경

    true= np.ones((batch_size,1)) # batch_size는 한 번에 몇개의 실제이미지와 가짜이미지를 판별자에 넣을지 결정
    #먼저 batch_size 만큼 MNIST 이미지를 랜덤하게 불러와 판별자에 집어 넣는 과정이다. 실제이미지를 입력햇으므로 모두 참(1)
    fake = np.zeros((batch_size,1))

    for i in range(epoch):
        idx= np.random.randint(0, x_train.shape[0], batch_size) # 실제 이미지를 랜덤하게 선택해 판별자에 입력
        imgs = x_train[idx]
        d_loss_real = discriminator.train_on_batch(imgs, true) # batch_size 만큼 판별 시작

        # 가상 이미지를 판별자에 입력하기
        noise = np.random.normal(0,1,(batch_size, 100)) # 잠재 공간에서 무작위로 샘플링
        gen_imgs = generator.predict(noise)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake) # gen_imgs에 모두 가짜(0)라는 레이블이 붙음

        # 판별자와 생성자의 오차를 계산
        d_loss = np.add(d_loss_real, d_loss_fake)*0.5
        g_loss = gan.train_on_batch(noise,true) # 판별자와 생성자를 연결해서 만든 gan모델을 이용해 생성자의 오차(g_loss)를 구함
        print('epoch:%d'%i,'d_loss=%.4f'%d_loss,' g_loss=%.4f'%g_loss) #생성자와 판별자의 오차 출력

        if i % saving_interval == 0:
            noise = np.random.normal(0,1,(25,100)) #잠재공간에서 무작위로 샘플링
            gen_imgs = generator.predict(noise)
            gen_imgs = gen_imgs*0.5+0.5 # rescale : 0~1

            fig, axs = plt.subplots(5,5)
            count = 0
            for j in range(5):
                for k in range(5):
                    axs[j,k].imshow(gen_imgs[count,:,:,0],cmap='gray')
                    axs[j,k].axis('off')
                    count +=1
                fig.savefig('gan_imgs/gmnist_%d.png'%i)

    #descriminator의 wait값을 주는 것 학습은 generator만 하는 것
    #이미지가 실제 이미지와 가까워질것
gan_train(4001,32,200)


In [ ]:
# 생성적 적대 신경망